In [1]:
!nvidia-smi

Fri Mar 25 06:17:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/tort-siamese
!mkdir -p data
!mkdir -p tmp
!mkdir -p checkpoints
!mkdir -p pretrained
!mkdir -p results
!pwd
!ls data

Mounted at /content/gdrive
/content/gdrive/MyDrive/tort-siamese
/content/gdrive/MyDrive/tort-siamese
 cc.th.300.vec	       play_deka_df.parquet
'filt_df (1).pickle'   play_deka_df.pkl
 filt_df.pickle        processed_torts20210321.pkl
 legal_clean.pkl       processed_torts20210412.pkl
 play_deka_df.par      supreme_cases.pkl


In [3]:
import json
from IPython.display import Image, display_png
import numpy as np
from numpy.random import seed
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import LabelEncoder
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, precision_score, recall_score, confusion_matrix
from collections import Counter
from tqdm import tqdm
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers , regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Bidirectional , LSTM, GlobalMaxPool1D
from tensorflow.keras.layers import Dropout,MaxPooling1D,Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Embedding
from tensorflow. keras.layers import Flatten, Dropout, Activation, Input, Dense, concatenate, GRU, Conv1D

In [4]:
print(tf.__version__)

2.8.0


### Inferences

In [6]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def microf1(y_true, y_pred):

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def macrof1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return tf.cast((y_true * K.square(y_pred)\
            + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0))),dtype=tf.float32)

def accuracy2(y_true, y_pred):
    return K.mean(tf.cast(tf.equal(y_true, tf.cast(y_pred < 0.5, dtype=tf.float32)), dtype=tf.float32))

In [7]:
def split_by_id(file = '../input/zero-shot-legal-03042021/filt_df.pickle'):
    filter_df = pd.read_pickle(file)
    train_indx, rest_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.30, n_splits=2, random_state = 42).split(filter_df, groups=filter_df['case_id']))
    rest_df = filter_df.iloc[rest_indx]
    dev_indx,test_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.50, n_splits=2, random_state = 42).split(rest_df, groups=rest_df['case_id']))

    train = filter_df.iloc[train_indx]
    dev = rest_df.iloc[dev_indx]
    test = rest_df.iloc[test_indx]
    
    def get_data_by_id(df):
        X = [] 
        Y = []
    
        for i,row in df.iterrows():
            x = row['plaintiff_token']
            x2 = row['legal_content_token']
            x4 = row['case_id']
            legal_encoded = row['legal_encoded']
            X.append([x, x2, legal_encoded, x4])
            Y.append(row['label'])
        X = np.array(X ,dtype=object)
        Y = np.array(Y)
        print('X:'+ str(X.shape),'Y:'+str(Y.shape),'legal_encoded_seq:' + str(len(X[:, 2])))
        return X,Y
    
    X_train,Y_train = get_data_by_id(train)
    X_dev,Y_dev = get_data_by_id(dev)
    X_test,Y_test = get_data_by_id(test)

    return X_train,Y_train,X_dev,Y_dev,X_test,Y_test

In [8]:
def split_by_id_2(file = '../input/zero-shot-legal-03042021/filt_df.pickle'):
    filter_df = pd.read_pickle(file)
    train_indx, rest_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.30, n_splits=2, random_state = 42).split(filter_df, groups=filter_df['case_id']))
    rest_df = filter_df.iloc[rest_indx]
    dev_indx,test_indx = next(sklearn.model_selection.GroupShuffleSplit(test_size=.50, n_splits=2, random_state = 42).split(rest_df, groups=rest_df['case_id']))

    train = filter_df.iloc[train_indx]
    dev = rest_df.iloc[dev_indx]
    test = rest_df.iloc[test_indx]
    
    def get_data_by_id(df):
        X = [] 
        Y = []
    
        for i,row in df.iterrows():
            x = row['plaintiff_token']
            x2 = row['legal_content_token']
            x3 = row['defendant_token']
            x4 = row['case_id']
            legal_encoded = row['legal_encoded']
            X.append([x, x2, x3,legal_encoded, x4])
            Y.append(row['label'])
        X = np.array(X ,dtype=object)
        Y = np.array(Y)
        print('X:'+ str(X.shape),'Y:'+str(Y.shape),'legal_encoded_seq:' + str(len(X[:, 3])))
        return X,Y
    
    X_train,Y_train = get_data_by_id(train)
    X_dev,Y_dev = get_data_by_id(dev)
    X_test,Y_test = get_data_by_id(test)

    return X_train,Y_train,X_dev,Y_dev,X_test,Y_test

In [9]:
def process_input_combined(num_words, X_train, X_dev, X_test):
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')
    X_combined = list(X_train[:, 0]) + list(X_train[:, 1])
    tokenizer.fit_on_texts(X_combined)

    #convert text data to numerical indexes
    train_seqs1 = tokenizer.texts_to_sequences(X_train[:, 0])
    train_seqs2 = tokenizer.texts_to_sequences(X_train[:, 1])
    
    dev_seqs1 = tokenizer.texts_to_sequences(X_dev[:, 0])
    dev_seqs2 = tokenizer.texts_to_sequences(X_dev[:, 1])
    
    test_seqs1 = tokenizer.texts_to_sequences(X_test[:, 0])
    test_seqs2 = tokenizer.texts_to_sequences(X_test[:, 1])
    
    max_len = max([len(x) for x in train_seqs1 + train_seqs2])

    train_seqs1 = tf.keras.preprocessing.sequence.pad_sequences(train_seqs1, maxlen=max_len, padding="post")
    test_seqs1=tf.keras.preprocessing.sequence.pad_sequences(test_seqs1, maxlen=max_len, padding="post")
    dev_seqs1 =tf.keras.preprocessing.sequence.pad_sequences(dev_seqs1, maxlen=max_len, padding="post")
    
    train_seqs2 = tf.keras.preprocessing.sequence.pad_sequences(train_seqs2, maxlen=max_len, padding="post")
    test_seqs2=tf.keras.preprocessing.sequence.pad_sequences(test_seqs2, maxlen=max_len, padding="post")
    dev_seqs2 =tf.keras.preprocessing.sequence.pad_sequences(dev_seqs2, maxlen=max_len, padding="post")

    return  train_seqs1, train_seqs2, dev_seqs1, dev_seqs2, test_seqs1, test_seqs2, max_len, tokenizer

In [10]:
def process_input(num_words, X_train, X_test, X_dev):
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')
    tokenizer.fit_on_texts(X_train)

    #convert text data to numerical indexes
    train_seqs = tokenizer.texts_to_sequences(X_train)
    dev_seqs = tokenizer.texts_to_sequences(X_dev)
    test_seqs = tokenizer.texts_to_sequences(X_test)

    max_sequnce_len = max([len(x) for x in train_seqs])

    train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=max_sequnce_len, padding="post")
    test_seqs=tf.keras.preprocessing.sequence.pad_sequences(test_seqs, maxlen=max_sequnce_len, padding="post")
    dev_seqs=tf.keras.preprocessing.sequence.pad_sequences(dev_seqs, maxlen=max_sequnce_len, padding="post")

    return  train_seqs, test_seqs, dev_seqs, max_sequnce_len, tokenizer

In [11]:
def process_input_combined_add_zero(num_words, X_train, X_dev, X_test, X_1, X_2):
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')
    X_combined = list(X_train[:, 0]) + list(X_train[:, 1])
    tokenizer.fit_on_texts(X_combined)

    #convert text data to numerical indexes
    train_seqs1 = tokenizer.texts_to_sequences(X_train[:, 0])
    train_seqs2 = tokenizer.texts_to_sequences(X_train[:, 1])
    
    dev_seqs1 = tokenizer.texts_to_sequences(X_dev[:, 0])
    dev_seqs2 = tokenizer.texts_to_sequences(X_dev[:, 1])
    
    test_seqs1 = tokenizer.texts_to_sequences(X_test[:, 0])
    test_seqs2 = tokenizer.texts_to_sequences(X_test[:, 1])
    

    zero_seq_1 = tokenizer.texts_to_sequences(X_1)
    zero_seq_2 = tokenizer.texts_to_sequences(X_2)

    max_len = max([len(x) for x in train_seqs1 + train_seqs2])

    train_seqs1 = tf.keras.preprocessing.sequence.pad_sequences(train_seqs1, maxlen=max_len, padding="post")
    test_seqs1=tf.keras.preprocessing.sequence.pad_sequences(test_seqs1, maxlen=max_len, padding="post")
    dev_seqs1 =tf.keras.preprocessing.sequence.pad_sequences(dev_seqs1, maxlen=max_len, padding="post")
    
    train_seqs2 = tf.keras.preprocessing.sequence.pad_sequences(train_seqs2, maxlen=max_len, padding="post")
    test_seqs2=tf.keras.preprocessing.sequence.pad_sequences(test_seqs2, maxlen=max_len, padding="post")
    dev_seqs2 =tf.keras.preprocessing.sequence.pad_sequences(dev_seqs2, maxlen=max_len, padding="post")


    zero_seq_1=tf.keras.preprocessing.sequence.pad_sequences(zero_seq_1, maxlen=max_len, padding="post")
    zero_seq_2=tf.keras.preprocessing.sequence.pad_sequences(zero_seq_2, maxlen=max_len, padding="post")

    return  train_seqs1, train_seqs2, dev_seqs1, dev_seqs2, test_seqs1, test_seqs2, max_len, tokenizer, zero_seq_1, zero_seq_2

In [12]:
idx = {251 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 55',
692 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 425',
704 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 438',
154 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 172',
135 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 142',
715 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 448',
688 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 421',
202 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 249',
100 : 'ประมวลกฎหมายวิธีพิจารณาความอาญา 46',
417 :  'ประมวลกฎหมายแพ่งและพาณิชย์ 1336'}

In [13]:
BATCH_SIZE=64
EPOCHS = 5
NUM_WORDS=8000
embedding_size = 300
lr = 3e-4

In [15]:
df = pd.read_parquet('data/play_deka_df.parquet')

In [16]:
df.head()

,case,token,long_text,law,law_content,law_content_token
0,2483###729,"[คดี, นี้, โจทก์, ฟ้อง, ขอ, ให้, ลง, โทษ, จำเล...",คดีนี้โจทก์ฟ้องขอให้ลงโทษจำเลยฐานมีอาวุธปืนโดย...,ประมวลกฎหมายอาญา 60,ผู้ใดเจตนาที่จะกระทำต่อบุคคลหนึ่ง แต่ผลของกา...,"[ , , ผู้, ใด, เจตนา, ที่, จะ, กระทำ, ต่อ, บุ..."
1,2483###729,"[คดี, นี้, โจทก์, ฟ้อง, ขอ, ให้, ลง, โทษ, จำเล...",คดีนี้โจทก์ฟ้องขอให้ลงโทษจำเลยฐานมีอาวุธปืนโดย...,ประมวลกฎหมายอาญา 249,ผู้ใดทำบัตรหรือโลหะธาตุอย่างใด ๆ ให้มีลักษณะ...,"[ , , ผู้, ใด, ทำ, บัตร, หรือ, โลหะ, ธาตุ, อย..."
2,2483###729,"[คดี, นี้, โจทก์, ฟ้อง, ขอ, ให้, ลง, โทษ, จำเล...",คดีนี้โจทก์ฟ้องขอให้ลงโทษจำเลยฐานมีอาวุธปืนโดย...,ประมวลกฎหมายอาญา 59,บุคคลจะต้องรับผิดในทางอาญาก็ต่อเมื่อได้กระทำ...,"[ , , บุคคล, จะ, ต้อง, รับผิด, ใน, ทาง, อาญา,..."
3,2483###816,"[คดี, นี้, ได้, ความ, ว่า, จำเลย, เอา, ไฟ, จุด...",คดีนี้ได้ความว่าจำเลยเอาไฟจุดเผาตูบของเจ้าทรัพ...,ประมวลกฎหมายอาญา 186,ผู้ใดทำให้เสียหาย ทำลาย ซ่อนเร้น เอาไปเสีย ห...,"[ , , ผู้, ใด, ทำ, ให้, เสียหาย, , ทำลาย, ,..."
4,2483###816,"[คดี, นี้, ได้, ความ, ว่า, จำเลย, เอา, ไฟ, จุด...",คดีนี้ได้ความว่าจำเลยเอาไฟจุดเผาตูบของเจ้าทรัพ...,ประมวลกฎหมายอาญา 189,ผู้ใดช่วยผู้อื่นซึ่งเป็นผู้กระทำความผิด หรือ...,"[ , , ผู้, ใด, ช่วย, ผู้, อื่น, ซึ่ง, เป็น, ผ..."


In [17]:
df2 = pd.read_pickle('data/filt_df.pickle')

In [18]:
exludes = set(df2.case_id)

In [19]:
'2483###729' in exludes

False

In [20]:
e = {}
for c, d in df2.iterrows():
  e[d.legal_encoded] = d.legal_content_token
len(e)

10

In [21]:
import random
random.randrange(10)
e.keys()

e[list(e)[random.randrange(10)]]

['ภาย',
 'ใน',
 'บังคับ',
 'แห่ง',
 'กฎหมาย',
 'เจ้าของ',
 'ทรัพย์สิน',
 'มี',
 'สิทธิ',
 'ใช้สอย',
 'และ',
 'จำหน่าย',
 'ทรัพย์สิน',
 'ของ',
 'ตน',
 'และ',
 'ได้',
 'ซึ่ง',
 'ดอก',
 'ผล',
 'แห่ง',
 'ทรัพย์สิน',
 'นั้น',
 'กับ',
 'ทั้ง',
 'มี',
 'สิทธิ',
 'ติดตาม',
 'และ',
 'เอา',
 'คืน',
 'ซึ่ง',
 'ทรัพย์สิน',
 'ของ',
 'ตน',
 'จาก',
 'บุคคล',
 'ผู้',
 'ไม่',
 'มี',
 'สิทธิ',
 'จะ',
 'ยึดถือ',
 'ไว้',
 'และ',
 'มี',
 'สิทธิ',
 'ขัดขวาง',
 'มิ',
 'ให้',
 'ผู้',
 'อื่น',
 'สอด',
 'เข้า',
 'เกี่ยวข้อง',
 'กับ',
 'ทรัพย์สิน',
 'นั้น',
 'โดย',
 'มิ',
 'ชอบ',
 'ด้วย',
 'กฎหมาย']

In [22]:
# law = df.law_content_token.apply(join)
X_1 = []
X_2 = []
X_3 = []
Y = []
for i, doc in df.iterrows():
  case = doc.case
  if case in exludes:
    continue
  if (doc.law_content_token is not None and len(doc.law_content_token) > 10):
    df3 = df[df['case'] == doc.case]
    token = ' '.join(doc.token)
    law_t = ' '.join(doc.law_content_token[1:])
    X_1.append(token)
    X_2.append(law_t)
    X_3.append(case)
    Y.append(1)
    law_t_2 = e[list(e)[random.randrange(10)]]
    X_1.append(token)
    X_2.append(law_t_2)
    X_3.append(case)
    Y.append(0)
    if (len(X_1) % 1000 == 0):
      print(len(X_1))
    if (len(X_1) == 200000):
      break
    # break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [23]:
df3

,case,token,long_text,law,law_content,law_content_token
108886,2541###5838,"[โจทก์, ฟ้อง, ว่า, , จำเลย, ทั้ง, สาม, ร่วม, ...",โจทก์ฟ้องว่า จำเลยทั้งสามร่วมกันลักเอานาฬิกาข้...,ประมวลกฎหมายอาญา 89,ถ้ามีเหตุส่วนตัวอันควรยกเว้นโทษ ลดโทษหรือเพิ...,"[ , , ถ้า, มี, เหตุ, ส่วน, ตัว, อัน, ควร, ยกเ..."
108887,2541###5838,"[โจทก์, ฟ้อง, ว่า, , จำเลย, ทั้ง, สาม, ร่วม, ...",โจทก์ฟ้องว่า จำเลยทั้งสามร่วมกันลักเอานาฬิกาข้...,ประมวลกฎหมายอาญา 295,ผู้ใดทำร้ายผู้อื่น จนเป็นเหตุให้เกิดอันตรายแ...,"[ , , ผู้, ใด, ทำร้าย, ผู้, อื่น, , จน, เป็น..."
108888,2541###5838,"[โจทก์, ฟ้อง, ว่า, , จำเลย, ทั้ง, สาม, ร่วม, ...",โจทก์ฟ้องว่า จำเลยทั้งสามร่วมกันลักเอานาฬิกาข้...,ประมวลกฎหมายอาญา 335,ผู้ใดลักทรัพย์\n(1) ในเวลากลางคืน\n(2) ในที่...,"[ , , ผู้, ใด, ลัก, ทรัพย์, \n, (, 1, ), , ใ..."
108889,2541###5838,"[โจทก์, ฟ้อง, ว่า, , จำเลย, ทั้ง, สาม, ร่วม, ...",โจทก์ฟ้องว่า จำเลยทั้งสามร่วมกันลักเอานาฬิกาข้...,ประมวลกฎหมายวิธีพิจารณาความอาญา 192,ห้ามมิให้พิพากษา หรือสั่ง เกินคำขอ หรือที่มิ...,"[ , , ห้าม, มิ, ให้, พิพากษา, , หรือ, สั่ง, ..."


In [24]:
len(X_1)

200000

In [25]:
#split data
X_train,Y_train,X_dev,Y_dev,X_test,Y_test = split_by_id(file = 'data/filt_df.pickle')

X:(17373, 4) Y:(17373,) legal_encoded_seq:17373
X:(3726, 4) Y:(3726,) legal_encoded_seq:3726
X:(3782, 4) Y:(3782,) legal_encoded_seq:3782


In [26]:
#process data
train_seqs1, train_seqs2, dev_seqs1, dev_seqs2, test_seqs1, test_seqs2, max_len, tokenizer, zero_seq_1, zero_seq_2 = process_input_combined_add_zero(NUM_WORDS, X_train, X_dev, X_test, X_1, X_2)

In [27]:
def evaluate_avg(y_pred, true):
  pred = [1 if lst > np.mean(y_pred) else 0  for lst in y_pred ]
  f1 =  f1_score(true, pred, labels=[1])
  recall = recall_score(true, pred,labels=[1])
  precision = precision_score(true,pred,labels=[1])
  print(f1, recall, precision)

In [28]:
cnn_base = tf.keras.models.load_model('saved_models/multi_simese_cnn', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
cnn_result = cnn_base.predict([zero_seq_1, zero_seq_2], verbose=1)

6250/6250 [==============================] - 67s 9ms/step


In [42]:
def evaluate_avg(y_pred, true):
  pred = [1 if lst > 0.52 else 0  for lst in y_pred ]
  f1 =  f1_score(true, pred, labels=[1], average='micro')
  recall = recall_score(true, pred, labels=[1], average='micro')
  precision = precision_score(true,pred, labels=[1], average='micro')
  print(f1, recall, precision)

In [30]:
len(cnn_result), len(Y)

(200000, 200000)

In [44]:
evaluate_avg(cnn_result, Y)

0.016505139500734213 0.00843 0.39209302325581397


In [32]:
np.mean(cnn_result)

0.27880725

In [46]:
lstm_base = tf.keras.models.load_model('saved_models/multi_simese_lstm', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
lstm_result = lstm_base.predict([zero_seq_1, zero_seq_2], verbose=1)

6250/6250 [==============================] - 5401s 864ms/step


In [47]:
evaluate_avg(lstm_result, Y)

0.1425484165904746 0.1029 0.23190300189308574


In [ ]:
cxx =np.array(Y)

In [ ]:
type(lstm_result), type(cxx)

In [ ]:
ccc = {
    'X3': X_3,
    'cnn_result': cnn_result,
    'lstm_result': lstm_result,
    'expected': Y
}

In [ ]:
np.save('cnn_result.npy', cnn_result)
np.save('lstm_result.npy', lstm_result)
np.save('expected.npy', Y)

In [ ]:
gru_base = tf.keras.models.load_model('saved_models/multi_simese_gru', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
gru_result = gru_base.predict([zero_seq_1, zero_seq_2], verbose=1)

In [ ]:
# np.save('gru_result.npy', gru_result)
evaluate_avg(gru_result, Y)

In [ ]:
bi_gru_base = tf.keras.models.load_model('saved_models/multi_simese_bi_gru', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
bi_gru_result = bi_gru_base.predict([zero_seq_1, zero_seq_2], verbose=1)

In [ ]:
len(bi_gru_result)

In [ ]:
# np.save('bi_gru_result.npy', bi_gru_result)
evaluate_avg(bi_gru_result, Y)

In [37]:
bi_lstm_base = tf.keras.models.load_model('saved_models/multi_simese_bi_lstm', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
bi_lstm_result = bi_lstm_base.predict([zero_seq_1, zero_seq_2], verbose=1)

6250/6250 [==============================] - 437s 69ms/step


In [45]:
evaluate_avg(bi_lstm_result, Y)

0.10487872105843439 0.06088 0.3782306163021869


In [41]:
np.mean(bi_lstm_result)

0.43211707

In [ ]:
np.save('bi_lstm_result.npy', bi_lstm_result)
evaluate_avg(bi_lstm_result, Y)

In [ ]:
evaluate_avg(bi_lstm_result, Y)

In [ ]:
X_train,Y_train,X_dev,Y_dev,X_test,Y_test_2 = split_by_id_2(file = 'data/filt_df.pickle')
train_seqs1, test_seqs1, dev_seqs1, max_sequence_len1, tokenizer1 = process_input(NUM_WORDS, X_train[:, 0], X_test[:, 0], X_dev[:, 0])
train_seqs2, test_seqs2, dev_seqs2, max_sequence_len2, tokenizer2 = process_input(NUM_WORDS, X_train[:, 1], X_test[:, 1], X_dev[:, 1])
train_seqs3, test_seqs3, dev_seqs3, max_sequence_len3, tokenizer3 = process_input(NUM_WORDS, X_train[:, 2], X_test[:, 2], X_dev[:, 2])

In [ ]:
three_encoders_sim_base = tf.keras.models.load_model('saved_models/three_encoders_sim', custom_objects={'recall':recall, 'precision':precision, 'microf1':microf1, 'macrof1': macrof1})
three_encoders_sim_result = three_encoders_sim_base.predict([test_seqs1,test_seqs2, test_seqs3], verbose=1)

In [ ]:
np.sum(Y_test == Y_test_2)

In [ ]:
len(cnn_result), len(Y_test), len(X_test[:, 2])

In [ ]:
pred_cnn = [x[0] for x in cnn_result]
pred_lstm = [x[0] for x in lstm_result]
pred_gru = [x[0] for x in gru_result]
pred_bi_gru = [x[0] for x in bi_gru_result]
pred_bi_lstm = [x[0] for x in bi_lstm_result]
pred_three_encoders_sim = [x[0] for x in three_encoders_sim_result]

In [ ]:
df = pd.DataFrame({
    "case": X_test[:, 4],
    "legal_encoded": X_test[:, 3],
    "pred_cnn": pred_cnn,
    "pred_lstm": pred_lstm,
    "pred_gru": pred_gru,
    "pred_bi_gru": pred_bi_gru,
    "pred_bi_lstm": pred_bi_lstm,
    "pred_three_encoders_sim": pred_three_encoders_sim,
    "expected": Y_test
})

In [ ]:
df.head()

In [ ]:
df.to_pickle('raw_result.pkl')

In [ ]:
df.to_csv('raw_result.csv')

In [ ]:
df

### Evaluate

In [ ]:
df = pd.read_csv('raw_result.csv')

In [ ]:
evaluation = evaluate(df, 'pred_cnn')
evaluation_at_3 = evaluate_at_k(df, 'pred_cnn', 3)
evaluation_at_5 = evaluate_at_k(df, 'pred_cnn', 5)
evaluation_at_10 = evaluate_at_k(df, 'pred_cnn', 10)

In [ ]:
evaluation

In [ ]:
evaluation_at_3

In [ ]:
evaluation_at_5

In [ ]:
evaluation_at_5

In [ ]:
pd.DataFrame(evaluation).T

In [ ]:
pd.DataFrame(evaluation_at_3).T

In [ ]:
idx = {251 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 55',
692 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 425',
704 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 438',
154 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 172',
135 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 142',
715 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 448',
688 : 'ประมวลกฎหมายแพ่งและพาณิชย์ 421',
202 : 'ประมวลกฎหมายวิธีพิจารณาความแพ่ง 249',
100 : 'ประมวลกฎหมายวิธีพิจารณาความอาญา 46',
417 :  'ประมวลกฎหมายแพ่งและพาณิชย์ 1336'}
models = ['pred_cnn', 'pred_lstm', 'pred_gru', 'pred_bi_gru', 'pred_bi_lstm', 'pred_three_encoders_sim']
K = [3]
evaluations = {}
for model in tqdm(models):
  for k in K:
    evaluation_at_k = evaluate_at_k(df, model, k)
    evaluations[f"{model}@{k}"] = evaluation_at_k

In [ ]:
pd.DataFrame(evaluations)

In [ ]:
evaluation_df = pd.DataFrame.from_dict({(i,j): evaluations[i][j] 
                           for i in evaluations.keys() 
                           for j in evaluations[i].keys()},
                       orient='index')

In [ ]:
evaluation_df.to_excel('evaluation.xlsx')

In [ ]:
move = 5
for model in list(evaluations.keys()):
  print(model)
  for id in idx.keys():
    evaluation = evaluations[model][id]
    print(evaluation['f1'])
    print(evaluation['precision'])
    print(evaluation['recall'])
    print()